In [3]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

Using TensorFlow backend.


In [2]:
#stop word removal
from nltk.corpus import stopwords
stop={'a', 'of', 'on', 'the'} #add related stop word here
print(stop)

{'of', 'a', 'on', 'the'}


In [39]:
# print(len(answers))
# print(len(answers_level))

# my_dict = { 'name' : answers,'age' : answers_level}
# df = pd.DataFrame(my_dict)
# df.to_csv('csv_example')

In [8]:
df_csv = pd.read_csv('datasets/only_cyberbyllying_formspring_datasets.csv')

In [5]:
df_csv1 = pd.read_csv('datasets/not_cyberbyllying_formspring_datasets.csv')

In [6]:
df_csv2 = pd.read_csv('datasets/ask_fm_data.csv')

In [7]:
df_csv2.head(10100)

,Unnamed: 0,q_a,label
0,0,how to tell if a guy is gay if they seem super...,0
1,1,Idk I'm not gay,0
2,2,This person is 100% fake so beware Isabella!,1
3,3,Oof lmao how is she fake? Who's this lmao shit...,1
4,4,If your loved one left n cut all ties w you sh...,0
...,...,...,...
9993,9993,Basically anything omg,0
9994,9994,Why is it that when I ask someone so random as...,0
9995,9995,Thats when my bitch mode comes out,0
9996,9996,Post a picture of your favorite celebrity!,0


In [9]:
#marge all data from files that has been read

q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(df_csv['label'][i])
    q_a.append(df_csv['q_a'][i])
print(len(q_a))
print(len(label))

for i in range(len(df_csv1)):
    label.append(df_csv1['label'][i])
    q_a.append(df_csv1['q_a'][i])
print(len(q_a))
print(len(label))

for i in range(len(df_csv2)):
    label.append(df_csv2['label'][i])
    q_a.append(df_csv2['q_a'][i])

print(len(q_a))
print(len(label))

1026
1026
12726
12726
22724
22724


In [10]:
print(df_csv2['label'][999])

0


In [12]:
print(q_a[99])

YOU SHOULD DIE


In [15]:
#this code will stem text
stem_both_q_a=[]
for i in q_a:
    ps = PorterStemmer() #stemming word removing
    words = str(i).split(' ') 
    s=''
    for w in words: 
        stem=ps.stem(w)
        s=s+' '+stem
    s=s.strip()

    stem_both_q_a.append(s)
q_a=stem_both_q_a

print(q_a)

["fuck justin bieber. he' a fuck fag", 'it also funni how u stalk my whole twitter! nice goin! bitch!', "hello bitch! how' ur day going? good I hope :) it' beauti and sunny!", 'hey. whi you such a bitch?', 'how old are u ? and how do you know michael?', 'love youu sexi thang(;', 'what wa your favorit toy to play with as a child?', 'who decid to take a shit on your face?', "whi do u follow the bandwagon? formspring. twitter. what' next gay justin beiber.", "lmao.. yurr reall funni skinni ass bitchh.. hm.. that isn't realli much of an insult now is it? what if i wa fat? lol u suck at talkn shit :] later white trash skank :] ur super ugli nd that guy u like realli isnt gonna come back around for u. laterrrr", 'lol how cute u have friend that back u up :] doesnt matter. ur still a fake bitch :] go find a real bf. not someon out of ur league. lol ugli ass', 'shit talkin bitch', 'still a bitch :]', 'suck it', 'whyy are youu suchh aa faggottt(:', 'your ugly.', "I have one for your inform wtf.

In [16]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=q_a
comments_attack['label']=label

In [17]:
dataframe = comments_attack

# Text preprocessing
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" y ", ' why '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" u ", ' you '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" w ", ' with '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" mf ", ' motherfucker '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDGAF ", ' i dont give a fuck '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" IDRC ", ' I Dont Really Care '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tf ", ' the fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wtf ", ' what the fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" yanno ", ' you know '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" igt ", ' i know right '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" r ", ' are '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tbh ", ' to be honest '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ik ", ' i know '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" af ", ' as fuck '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" des ", ' this '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bout ", ' about '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" em ", ' them '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" stg ", ' swear to god '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" bj ", ' blow job '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" ig ", ' i guess '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fagg ", ' faggot '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" fag ", ' faggot '))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" tfw ", ' That Feel When '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" wa ", ' was '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" n ", ' and '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" y0 ", ' you '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" dat ", ' that '))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(" YO ", ' you '))



In [19]:
#this code will remove stop word
for c in stop: #stop word removing
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(' '+c+' ', ' '))


In [20]:
print(dataframe['label'])

0        1
1        1
2        1
3        1
4        1
        ..
22719    0
22720    0
22721    0
22722    0
22723    0
Name: label, Length: 22724, dtype: int64


In [21]:
comments_attack['label'][990]==0

False

In [22]:
print(dataframe['comment'])

0                           fuck justin bieber he fuck fag
1        it also funni how you stalk my whole twitter n...
2        hello bitch how ur day going good i hope it be...
3                                   hey whi you such bitch
4              how old are you and how do you know michael
                               ...                        
22719                                      basic anyth omg
22720    whi is it that when i ask someon so random ass...
22721                     that when my bitch mode come out
22722                   post pictur your favorit celebrity
22723                                       hunter hay omg
Name: comment, Length: 22724, dtype: object


In [23]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.01, random_state=42)

In [24]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  22496
test comments length:  228


In [25]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(dataframe['comment'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [26]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

#feartures nlp
# dataframe['char_count'] = dataframe['comment'].apply(len)
# dataframe['word_count'] = dataframe['comment'].apply(lambda x: len(x.split()))
# dataframe['word_density'] = dataframe['char_count'] / (dataframe['word_count']+1)
# dataframe['punctuation_count'] = dataframe['comment'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
# dataframe['title_word_count'] = dataframe['comment'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
# dataframe['upper_case_word_count'] = dataframe['comment'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))


In [27]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [28]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [30]:
# Naive Bayes on Count Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
a=naive_bayes.MultinomialNB()
a.fit(xtrain_count, train_y)
print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))
trans=count_vect.transform(['fuck your son'])

predictions = a.predict(trans)
print(predictions)

NB, Count Vectors:   accuracy: 0.9342105263157895      f1 score: 0.9230553181096592
[0]


In [46]:
clf = Pipeline([
    ('vect',count_vect),
    ('clf', naive_bayes.MultinomialNB()),
])

In [47]:
clf = clf.fit(train_x, train_y)

In [48]:
clf = Pipeline([
    ('vect',count_vect),
    ('clf', naive_bayes.MultinomialNB()),
])
clf = clf.fit(train_x, train_y)
# correctly classify nasty comment

In [50]:
#check predictions
for i in valid_x:
    predictions = clf.predict([i])
    if predictions[0]==1:
        
        # accuracy = metrics.accuracy_score(predictions, ['i wanna fuck you'])
        # f1score = metrics.f1_score(['i wanna fuck you'], predictions,average='weighted')
        print(predictions)
        print(i)

[1]
if you say so cau you know all black guy right
[1]
yesssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss4 theee lasttt timeee ask him httpwwwformspringmetruac
[1]
are you confid with your dick size you have small dick
[1]
bruh get fuck outta here with your dumb ass
[1]
fuck you bithc
[1]
sinc you know everyth you know your an asshol
[1]
my ass look fuckabl for sure
[1]
yhuuu readi to get yhur beat thi saturdayy


In [51]:
# Naive Bayes on Count Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
print("NB, Count Vectors:   accuracy: %s      f1 score: %s"% (accuracy,f1score))
# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, Count Vectors:   accuracy: 0.9342105263157895      f1 score: 0.9230553181096592
NB, WordLevel TF-IDF:   accuracy: 0.9342105263157895     f1 score: 0.9148301788955148
NB, N-Gram Vectors:   accuracy: 0.9342105263157895     f1 score: 0.9093701071901722
NB, CharLevel Vectors:   accuracy: 0.956140350877193   f1 score: 0.9475105588044184


In [52]:
# Linear Classifier on Count Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count, valid_y)
print("LR, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR, Count Vectors:   accuracy: 0.9254385964912281   f1 score: 0.9034742027482501
LR, WordLevel TF-IDF:   accuracy: 0.9649122807017544   f1 score: 0.9613988241632118
LR, N-Gram Vectors:   accuracy: 0.9254385964912281   f1 score: 0.8896015665587659


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LR, CharLevel Vectors:   accuracy: 0.956140350877193   f1 score: 0.9498007092866333


In [53]:
# SVM Classifier on Count Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count,valid_y)
print("SVM, Count Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("SVM, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("SVM, N-Gram Vectors TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# SVM Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("SVM, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, Count Vectors:   accuracy: 0.9210526315789473   f1 score: 0.8874068733477529


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, WordLevel TF-IDF:   accuracy: 0.9254385964912281   f1 score: 0.8896015665587659


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, N-Gram Vectors TF-IDF:   accuracy: 0.9254385964912281   f1 score: 0.8896015665587659
SVM, CharLevel Vectors:   accuracy: 0.9254385964912281   f1 score: 0.8896015665587659


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [54]:
# Feed forward NN with 1 hidden layer
def model_FF(xtrain, ytrain, xvalid, yvalid, hidden_size, epochs =1):
    # create input layer 
    input_layer = layers.Input((xtrain.shape[1], ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(hidden_size, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(3, activation="softmax")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    classifier.fit(xtrain, ytrain,
                  batch_size=256,
                  epochs=epochs,
                  shuffle = True)
    # scores of the classifier
    predictions = classifier.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = classifier.evaluate(xvalid, yvalid, verbose=0)
#     f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)



In [55]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [493]:
# # NN Classifier on Count Vectors
# accuracy, f1score = model_FF(xtrain_count, train_y_onehot, xvalid_count, valid_y_onehot, 100)
# print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

# NN Classifier on Word Level TF IDF Vectors
# accuracy, f1score = model_FF(xtrain_tfidf, train_y_onehot, xvalid_tfidf, valid_y_onehot, 100)
# print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

# # NN Classifier on Ngram Level TF IDF Vectors
# accuracy, f1score = model_FF(xtrain_tfidf_ngram, train_y_onehot, xvalid_tfidf_ngram, valid_y_onehot, 100)
# print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

# NN Classifier on Count Vectors
# accuracy, f1score = model_FF(xtrain_tfidf_ngram_chars, train_y_onehot, xvalid_tfidf_ngram_chars, valid_y_onehot, 100)
# print("NN, Count Vectors accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

In [56]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [57]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
22496/22496 [==============================] - 6s 286us/step - loss: 0.3671 - accuracy: 0.9027
Epoch 2/3
22496/22496 [==============================] - 6s 284us/step - loss: 0.2059 - accuracy: 0.9243
Epoch 3/3
22496/22496 [==============================] - 6s 287us/step - loss: 0.1736 - accuracy: 0.9337
CNN, Word Embeddings acuuracy accuracy:0.9605262875556946     f1 score: 0.9573545225415148


In [59]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(3, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [60]:
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
22496/22496 [==============================] - 25s 1ms/step - loss: 0.3454 - accuracy: 0.9020
Epoch 2/3
22496/22496 [==============================] - 27s 1ms/step - loss: 0.2162 - accuracy: 0.9214
Epoch 3/3
22496/22496 [==============================] - 27s 1ms/step - loss: 0.1980 - accuracy: 0.9255
LSTM, Word Embeddings accuracy:0.9473684430122375     f1 score: 0.937012670565302


In [61]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print(accuracy)

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
